In [1]:
import numpy as np
from tqdm import tqdm
import os
import json
from torch.utils.data import Dataset, Sampler
import torch.utils.data as data

/home/mdelabrassinne/anaconda3/envs/envDeepLearning/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
path = "/scratch/users/mdelabrassinne/Database/england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/2_baidu_soccer_embeddings.npy"

ar = np.load(path)

In [13]:
ar.shape

(2700, 8576)

In [4]:
print(ar[:100, :100])

[[0.38118884 0.02770964 0.00367757 ... 0.17223546 0.09156295 0.02553807]
 [0.42885312 0.01224316 0.         ... 0.20666476 0.17031477 0.12626918]
 [0.2583254  0.00581192 0.02311912 ... 0.282737   0.14367244 0.10988639]
 ...
 [0.04447667 0.3821194  0.1784471  ... 0.01555356 0.04229367 0.35579708]
 [0.01905869 0.26908693 0.23427132 ... 0.1827074  0.12168921 0.61314577]
 [0.00867339 0.18794657 0.29181525 ... 0.31754282 0.14958553 0.5993214 ]]


In [5]:
path = '/scratch/users/mdelabrassinne/Database/'

In [52]:
features =[]
jsonList = []
for root, dirs, files in tqdm(os.walk(path)):
    #print(files)
    for file in files:
        if file.endswith("baidu_soccer_embeddings.npy"):
            features.append(os.path.join(root, file))
        if file.endswith(".json"):
            jsonList.append(os.path.join(root, file))

            
data = list(zip(features, jsonList))

98714it [00:59, 1649.02it/s]


In [53]:
print(len(features))
print(np.load(features[0]))
print(data[0])

f = open(data[0][1])
d = json.load(f)
print(d['annotations'][0]['label'])
print(d['annotations'][1]['gameTime'])
print(d['annotations'][1]['gameTime'][4:6])
print(d['annotations'][1]['gameTime'][7:9])

print(len(d['annotations']))

1100
[[3.8118884e-01 2.7709639e-02 3.6775670e-03 ... 3.6195982e-03
  0.0000000e+00 2.9661678e-04]
 [4.2885312e-01 1.2243160e-02 0.0000000e+00 ... 3.3716105e-03
  0.0000000e+00 1.4415581e-04]
 [2.5832540e-01 5.8119199e-03 2.3119124e-02 ... 3.9737178e-03
  0.0000000e+00 1.3890492e-05]
 ...
 [5.2272514e-03 2.9039851e-01 2.2069754e-01 ... 3.0468414e-03
  0.0000000e+00 7.2249270e-05]
 [3.9711028e-02 2.4259840e-01 2.3015372e-01 ... 3.1011349e-03
  0.0000000e+00 3.6978102e-04]
 [5.4142091e-02 2.1650633e-01 1.8599744e-01 ... 1.7698978e-03
  0.0000000e+00 5.4636848e-04]]
('/scratch/users/mdelabrassinne/Database/england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/1_baidu_soccer_embeddings.npy', '/scratch/users/mdelabrassinne/Database/england_epl/2014-2015/2015-02-21_-_18-00_Chelsea_1_-_1_Burnley/Labels-v2.json')
Kick-off
1 - 02:13
02
13
190


In [ ]:
class BaiduDataset(Dataset):    #modifiable
    """
    Class to prepare the data for a neural network in pyTorch. The database is composed of 3 tables:
    - Video: to save the video information
    - Sequence: to save the sequence information labelled with the soccer actions and record the sound of the sequence
    - Image: to save the image information of the sequence
    """
    def __init__(self, data_list, transform=None):
        self.data_list = data_list
        self.transform = transform
        
    def __len__(self):
        return len(self.data_list)
    
    def __getitem__(self, idx):
        feat_path = self.data_list[idx][0]       # path towards Baidu features
        label_path = self.data_list[idx][1]      # path towards associated json
        
        features = np.load(feat_path)
        
        
        if self.transform:
            proba = 1 / (len(self.transform))
            val = random.random()
            current = 0
            for i in range(len(self.transform)):
                if val < current + proba:
                    image = self.transform[i](image)
                    return image, label
            return self.transform[-1](image), label
        else:                
            return image, label

In [78]:
features =[]
labels = []
timeLabel = []

path = '/scratch/users/mdelabrassinne/Database/'


for root, dirs, files in tqdm(os.walk(path)):
    #print(files)
    for file in files:
        if file.endswith("baidu_soccer_embeddings.npy"):
            features.append(os.path.join(root, file))
            print(os.path.join(root, file))
            
        if file.endswith("Labels-v2.json"):
            jsonPath =(os.path.join(root, file))
            print(jsonPath)
            f = open(jsonPath)
            d = json.load(f)
            
            try:
                #print('test')
                lab = [d['annotations'][i]['label'] for i in range(len(d['annotations']))]         # 1fps, don't know if keep 
                time = [60 * int(d['annotations'][i]['gameTime'].split(':')[0][-2:]) + int(d['annotations'][i]['gameTime'].split(':')[1][:2]) for i in range(len(d['annotations']))]
            
            except:
                #print(d)
                lab = [d['predictions'][i]['label'] for i in range(len(d['predictions']))]         # 1fps, don't know if keep 
                time = [60 * int(d['predictions'][i]['gameTime'].split(':')[0][-2:]) + int(d['predictions'][i]['gameTime'].split(':')[1][:2]) for i in range(len(d['predictions']))]
                
                #print("\n\n")
            labels.append(lab)
            timeLabel.append(time)
            

            
data_l = list(zip(features, labels, timeLabel))

260it [00:00, 1304.25it/s]

/scratch/users/mdelabrassinne/Database/england_epl/2014-2015/2015-02-21_-_18-00_Chelsea_1_-_1_Burnley/Labels-v2.json
/scratch/users/mdelabrassinne/Database/england_epl/2014-2015/2015-02-21_-_18-00_Crystal_Palace_1_-_2_Arsenal/Labels-v2.json


647it [00:00, 1267.79it/s]

/scratch/users/mdelabrassinne/Database/england_epl/2014-2015/2015-02-21_-_18-00_Swansea_2_-_1_Manchester_United/Labels-v2.json
/scratch/users/mdelabrassinne/Database/england_epl/2014-2015/2015-02-22_-_19-15_Southampton_0_-_2_Liverpool/Labels-v2.json


908it [00:00, 1286.64it/s]

...

39599it [00:30, 1302.23it/s]

/scratch/users/mdelabrassinne/Database/france_ligue-1/2016-2017/2017-03-12_-_23-00_Lorient_1_-_2_Paris_SG/Labels-v2.json
/scratch/users/mdelabrassinne/Database/france_ligue-1/2016-2017/2017-03-19_-_23-00_Paris_SG_2_-_1_Lyon/Labels-v2.json


39824it [00:30, 1311.37it/s]


/scratch/users/mdelabrassinne/Database/france_ligue-1/2016-2017/2017-04-14_-_21-45_Angers_0_-_2_Paris_SG/Labels-v2.json


KeyboardInterrupt: 

In [79]:
features =[]
labels = []
timeLabel = []

path = '/scratch/users/mdelabrassinne/Database/'


for root, dirs, files in tqdm(os.walk(path)):
    #print(files)
    for file in files:
            
        if file.endswith("Labels-v2.json"):
            
            p = os.path.dirname(os.path.join(root, file))
            dire = os.path.basename(p).replace('_', ' ')
            p = os.path.join(os.path.dirname(p), dire)
            
            for r, d, fs in os.walk(p):
                for f in fs:
                    if f.endswith("baidu_soccer_embeddings.npy"):
                        p_current = os.path.join(r, f)
                        features.append(p_current)
                        print(p_current)

            
            jsonPath =(os.path.join(root, file))
            #print(jsonPath)
            f = open(jsonPath)
            d = json.load(f)
            
            print(jsonPath)
            
            try:
                #print('test')
                lab = [d['annotations'][i]['label'] for i in range(len(d['annotations'])) if d['annotations'][i]['gameTime'][0] ==  '1']         # 1fps, don't know if keep 
                time = [60 * int(d['annotations'][i]['gameTime'].split(':')[0][-2:]) + int(d['annotations'][i]['gameTime'].split(':')[1][:2]) for i in range(len(d['annotations'])) if d['annotations'][i]['gameTime'][0] ==  '1']
                #print([d['annotations'][i]['gameTime'] for i in range(len(d['annotations']))])
                
                labels.append(lab)
                timeLabel.append(time)
                
                lab = [d['annotations'][i]['label'] for i in range(len(d['annotations'])) if d['annotations'][i]['gameTime'][0] ==  '2']         # 1fps, don't know if keep 
                time = [60 * int(d['annotations'][i]['gameTime'].split(':')[0][-2:]) + int(d['annotations'][i]['gameTime'].split(':')[1][:2]) for i in range(len(d['annotations'])) if d['annotations'][i]['gameTime'][0] ==  '2']
                
                labels.append(lab)
                timeLabel.append(time)
                
            except:
                #print(d)
                lab = [d['predictions'][i]['label'] for i in range(len(d['predictions'])) if d['predictions'][i]['gameTime'][0] == '1']         # 1fps, don't know if keep 
                time = [60 * int(d['predictions'][i]['gameTime'].split(':')[0][-2:]) + int(d['predictions'][i]['gameTime'].split(':')[1][:2]) for i in range(len(d['predictions'])) if d['predictions'][i]['gameTime'][0] == '1']
                
                labels.append(lab)
                timeLabel.append(time)
                
                lab = [d['predictions'][i]['label'] for i in range(len(d['predictions'])) if d['predictions'][i]['gameTime'][0] == '2']         # 1fps, don't know if keep 
                time = [60 * int(d['predictions'][i]['gameTime'].split(':')[0][-2:]) + int(d['predictions'][i]['gameTime'].split(':')[1][:2]) for i in range(len(d['predictions'])) if d['predictions'][i]['gameTime'][0] == '2']
                
                labels.append(lab)
                timeLabel.append(time)
            

            
data_l = list(zip(features, labels, timeLabel))

270it [00:00, 1325.86it/s]

/scratch/users/mdelabrassinne/Database/england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/1_baidu_soccer_embeddings.npy
/scratch/users/mdelabrassinne/Database/england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/2_baidu_soccer_embeddings.npy
/scratch/users/mdelabrassinne/Database/england_epl/2014-2015/2015-02-21_-_18-00_Chelsea_1_-_1_Burnley/Labels-v2.json
/scratch/users/mdelabrassinne/Database/england_epl/2014-2015/2015-02-21 - 18-00 Crystal Palace 1 - 2 Arsenal/1_baidu_soccer_embeddings.npy
/scratch/users/mdelabrassinne/Database/england_epl/2014-2015/2015-02-21 - 18-00 Crystal Palace 1 - 2 Arsenal/2_baidu_soccer_embeddings.npy
/scratch/users/mdelabrassinne/Database/england_epl/2014-2015/2015-02-21_-_18-00_Crystal_Palace_1_-_2_Arsenal/Labels-v2.json


676it [00:00, 1340.73it/s]

/scratch/users/mdelabrassinne/Database/england_epl/2014-2015/2015-02-21 - 18-00 Swansea 2 - 1 Manchester United/1_baidu_soccer_embeddings.npy
/scratch/users/mdelabrassinne/Database/england_epl/2014-2015/2015-02-21 - 18-00 Swansea 2 - 1 Manchester United/2_baidu_soccer_embeddings.npy
/scratch/users/mdelabrassinne/Database/england_epl/2014-2015/2015-02-21_-_18-00_Swansea_2_-_1_Manchester_United/Labels-v2.json
/scratch/users/mdelabrassinne/Database/england_epl/2014-2015/2015-02-22 - 19-15 Southampton 0 - 2 Liverpool/1_baidu_soccer_embeddings.npy
/scratch/users/mdelabrassinne/Database/england_epl/2014-2015/2015-02-22 - 19-15 Southampton 0 - 2 Liverpool/2_baidu_soccer_embeddings.npy
/scratch/users/mdelabrassinne/Database/england_epl/2014-2015/2015-02-22_-_19-15_Southampton_0_-_2_Liverpool/Labels-v2.json


953it [00:00, 1346.06it/s]

...

98059it [01:12, 1430.87it/s]

/scratch/users/mdelabrassinne/Database/spain_laliga/2016-2017/2017-04-08 - 21-45 Malaga 2 - 0 Barcelona/1_baidu_soccer_embeddings.npy
/scratch/users/mdelabrassinne/Database/spain_laliga/2016-2017/2017-04-08 - 21-45 Malaga 2 - 0 Barcelona/2_baidu_soccer_embeddings.npy
/scratch/users/mdelabrassinne/Database/spain_laliga/2016-2017/2017-04-08_-_21-45_Malaga_2_-_0_Barcelona/Labels-v2.json
/scratch/users/mdelabrassinne/Database/spain_laliga/2016-2017/2017-04-26 - 20-30 Barcelona 7 - 1 Osasuna/1_baidu_soccer_embeddings.npy
/scratch/users/mdelabrassinne/Database/spain_laliga/2016-2017/2017-04-26 - 20-30 Barcelona 7 - 1 Osasuna/2_baidu_soccer_embeddings.npy
/scratch/users/mdelabrassinne/Database/spain_laliga/2016-2017/2017-04-26_-_20-30_Barcelona_7_-_1_Osasuna/Labels-v2.json


98493it [01:12, 1421.61it/s]

/scratch/users/mdelabrassinne/Database/spain_laliga/2016-2017/2016-08-20 - 19-15 Barcelona 6 - 2 Betis/1_baidu_soccer_embeddings.npy
/scratch/users/mdelabrassinne/Database/spain_laliga/2016-2017/2016-08-20 - 19-15 Barcelona 6 - 2 Betis/2_baidu_soccer_embeddings.npy
/scratch/users/mdelabrassinne/Database/spain_laliga/2016-2017/2016-08-20_-_19-15_Barcelona_6_-_2_Betis/Labels-v2.json
/scratch/users/mdelabrassinne/Database/spain_laliga/2016-2017/2016-08-27 - 21-15 Real Madrid 2 - 1 Celta Vigo/1_baidu_soccer_embeddings.npy
/scratch/users/mdelabrassinne/Database/spain_laliga/2016-2017/2016-08-27 - 21-15 Real Madrid 2 - 1 Celta Vigo/2_baidu_soccer_embeddings.npy
/scratch/users/mdelabrassinne/Database/spain_laliga/2016-2017/2016-08-27_-_21-15_Real_Madrid_2_-_1_Celta_Vigo/Labels-v2.json


98714it [01:12, 1362.46it/s]


In [83]:
print(len(data))
print(data[0])

500
('/scratch/users/mdelabrassinne/Database/england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/1_baidu_soccer_embeddings.npy', ['Kick-off', 'Ball out of play', 'Throw-in', 'Ball out of play', 'Corner', 'Ball out of play', 'Throw-in', 'Shots on target', 'Ball out of play', 'Corner', 'Offside', 'Shots on target', 'Offside', 'Ball out of play', 'Throw-in', 'Ball out of play', 'Throw-in', 'Shots off target', 'Ball out of play', 'Clearance', 'Ball out of play', 'Throw-in', 'Ball out of play', 'Throw-in', 'Ball out of play', 'Throw-in', 'Ball out of play', 'Corner', 'Ball out of play', 'Clearance', 'Ball out of play', 'Goal', 'Kick-off', 'Shots on target', 'Ball out of play', 'Corner', 'Ball out of play', 'Throw-in', 'Ball out of play', 'Corner', 'Ball out of play', 'Corner', 'Shots off target', 'Clearance', 'Ball out of play', 'Clearance', 'Ball out of play', 'Clearance', 'Ball out of play', 'Throw-in', 'Ball out of play', 'Throw-in', 'Shots on target', 'Foul', 'Shots off targe

In [87]:
class BaiduDataset(Dataset):    #modifiable
    """
    Class to prepare the data for a neural network in pyTorch. The database is composed of 3 tables:
    - Video: to save the video information
    - Sequence: to save the sequence information labelled with the soccer actions and record the sound of the sequence
    - Image: to save the image information of the sequence
    """
    def __init__(self, data_list, transform=None):
        self.data_list = data_list
        self.transform = transform
        
    def __len__(self):
        return len(self.data_list)
    
    def __getitem__(self, idx):
        feat_path = self.data_list[idx][0]       # path towards Baidu features
        label_list = self.data_list[idx][1]      # list of labels
        time_list = self.data_list[idx][2]       # list of timecodes
        
        features = np.load(feat_path)
        
        
        return features, label_list, time_list

In [80]:
cL = ['Corner', 'Goal', 'Penalty', 'Kick-off', 'Yellow card', 'Red card', 'Yellow->red card']

def getTrueCoef(label, timeCodes, T, nbClasses, r_c = 3, f = 1, r_d = 6):
    C = []      #confidence as a 2D np array instead of a list of tupes

    res_conf = np.zeros((T, nbClasses))
    res_disp = np.zeros((T, nbClasses))
    
    dictClasses = {}

    for c in range(nbClasses):
        dictClasses[c] = []


    for i in range(len(timeCodes)):        #loop over obtained labels
        t = timeCodes[i]
        c = label[i]
        
        if(c in cL):                           # focus on wanted classes
            c = cL.index(label[i])

            tmin = t - r_c * f
            tmax = t + r_c * f
            inds = np.arange(tmin, tmax+1)
            res_conf[inds, c] = 1
            
            dictClasses[c].append(t)
            
        if(t > T):
            print('\n\naie => t = {} -- T = {}\n\n'.format(t, T))
            
        if(i == 2700):
            print("possible issue ")

    for c in range(nbClasses):
        ind = np.array(dictClasses[c])
        if(len(ind) > 0):
            for t in range(T):

                try:
                    closestInd = np.argmin(np.abs(ind - t))         #take the one that is the most on the left if draw
                except:
                    print(ind)
                    print(t)
                    print(len(ind))

                if(np.abs(t - ind[closestInd]) <= r_d * f):
                    res_disp[t, c] = t - ind[closestInd]
            
        
    return res_conf, res_disp

test_C, test_D = getTrueCoef(data_l[0][1],data_l[0][2], np.load(data_l[0][0]).shape[0], len(cL) )

print(np.where(test_C != 0))
print(np.where(test_D != 0))

print(test_C.shape)
print(test_D.shape)

(array([   0,    1,    2,    3,  179,  180,  181,  182,  183,  184,  185,
        289,  290,  291,  292,  293,  294,  295,  664,  665,  666,  667,
        668,  669,  670,  787,  788,  789,  790,  791,  792,  793,  840,
        841,  842,  843,  844,  845,  846,  877,  878,  879,  880,  881,
        882,  883,  968,  969,  970,  971,  972,  973,  974, 1028, 1029,
       1030, 1031, 1032, 1033, 1034, 2018, 2019, 2020, 2021, 2022, 2023,
       2024, 2405, 2406, 2407, 2408, 2409, 2410, 2411, 2542, 2543, 2544,
       2545, 2546, 2547, 2548, 2697, 2698, 2699]), array([3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 4, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3]))
(array([   1,    2,    3,    4,    5,    6,  176,  177,  178,  179,  180,
        181,  183,  184,  185,  186,  187,  188,  286,  287,  288,  289,
        

In [177]:
cL = ['Corner', 'Goal', 'Penalty', 'Kick-off', 'Yellow card', 'Red card', 'Yellow->red card']

class BaiduDataset(Dataset):    #modifiable
    """
    Class to prepare the data for a neural network in pyTorch. The database is composed of 3 tables:
    - Video: to save the video information
    - Sequence: to save the sequence information labelled with the soccer actions and record the sound of the sequence
    - Image: to save the image information of the sequence
    """
    def __init__(self, data_list, transform=None):
        self.data_list = data_list
        self.transform = transform
        self.seq_length = 112
        
    def __len__(self):
        return len(self.data_list)
    
    def __getitem__(self, idx):
        feat_path = self.data_list[idx][0]       # path towards Baidu features
        label_list = self.data_list[idx][1]      # list of labels
        time_list = self.data_list[idx][2]       # list of timecodes

        features = np.load(feat_path)
        
        print(features.shape)
        indStart = np.ceil(np.random.uniform(0, features.shape[0]-self.seq_length))
        ind = np.array([True if (i >= indStart and i < indStart + self.seq_length) else False for i in range(features.shape[0])])
        
        print(features[ind,:].shape)
        print(label_list)
        print(time_list)
        print("trsfndsjn")
        return features[ind,:], label_list, time_list   #not of equal size => issue

In [81]:
cL = ['Corner', 'Goal', 'Penalty', 'Kick-off', 'Yellow card', 'Red card', 'Yellow->red card']

class BaiduDataset(Dataset):    #modifiable
    """
    Class to prepare the data for a neural network in pyTorch. The database is composed of 3 tables:
    - Video: to save the video information
    - Sequence: to save the sequence information labelled with the soccer actions and record the sound of the sequence
    - Image: to save the image information of the sequence
    """
    def __init__(self, data_list, transform=None):
        self.data_list = data_list
        self.transform = transform
        self.seq_length = 112
        
    def __len__(self):
        return len(self.data_list)
    
    def __getitem__(self, idx):
        feat_path = self.data_list[idx][0]       # path towards Baidu features
        label_list = self.data_list[idx][1]      # list of labels
        time_list = self.data_list[idx][2]       # list of timecodes

        features = np.load(feat_path)
        
        print(features.shape)
        indStart = np.ceil(np.random.uniform(0, features.shape[0]-self.seq_length))
        ind = np.array([True if (i >= indStart and i < indStart + self.seq_length) else False for i in range(features.shape[0])])
        
        
        coef_c, coef_d = getTrueCoef(label_list, time_list, features.shape[0], len(cL) )
        print(coef_c.shape)
        print(coef_d.shape)
        print("\n")
        return (features[ind,:], coef_c[ind, :], coef_d[ind, :])   #not of equal size => issue

In [63]:
cL = ['Corner', 'Goal', 'Penalty', 'Kick-off', 'Yellow card', 'Red card', 'Yellow->red card']

class BaiduDataset2(Dataset):    #modifiable
    """
    Class to prepare the data for a neural network in pyTorch. The database is composed of 3 tables:
    - Video: to save the video information
    - Sequence: to save the sequence information labelled with the soccer actions and record the sound of the sequence
    - Image: to save the image information of the sequence
    """
    def __init__(self, data_list, transform=None):
        self.data_list = data_list
        self.transform = transform
        self.seq_length = 112
        
    def __len__(self):
        return len(self.data_list)
    
    def __getitem__(self, idx):
        feat_path = self.data_list[idx][0]       # path towards Baidu features
        label_list = self.data_list[idx][1]      # list of labels
        time_list = self.data_list[idx][2]       # list of timecodes

        features = np.load(feat_path)
        
        #print(features.shape)
        indStart = np.ceil(np.random.uniform(0, features.shape[0]-self.seq_length))
        ind = np.array([True if (i >= indStart and i < indStart + self.seq_length) else False for i in range(features.shape[0])])
        
        
        
        lab, t = getTrueCoef(label_list, time_list, features.shape[0], len(cL) )
        return (feat_path, label_list, time_list)   #not of equal size => issue

In [170]:
idx = 10
feat_path = data_l[idx][0]       # path towards Baidu features
label_list = data_l[idx][1]      # list of labels
time_list = data_l[idx][2]       # list of timecodes
features = np.load(feat_path)

print(len(time_list))
print(len(label_list))
print(features.shape[0])

181
181
2700


In [84]:
dataLearning = BaiduDataset(data_l)
loaderLearning = data.DataLoader(dataLearning, batch_size = 20, shuffle= True, num_workers = 1)
feat, lab, t = next(iter(loaderLearning))

(2700, 8576)
(2700, 7)
(2700, 7)


(2700, 8576)
(2700, 7)
(2700, 7)


(2700, 8576)
(2700, 7)
(2700, 7)


(2700, 8576)
(2700, 7)
(2700, 7)


(2700, 8576)
(2700, 7)
(2700, 7)


(2700, 8576)
(2700, 7)
(2700, 7)


(2700, 8576)
(2700, 7)
(2700, 7)


(2919, 8576)


IndexError: Caught IndexError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/mdelabrassinne/anaconda3/envs/envDeepLearning/lib/python3.9/site-packages/torch/utils/data/_utils/worker.py", line 302, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/mdelabrassinne/anaconda3/envs/envDeepLearning/lib/python3.9/site-packages/torch/utils/data/_utils/fetch.py", line 58, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/mdelabrassinne/anaconda3/envs/envDeepLearning/lib/python3.9/site-packages/torch/utils/data/_utils/fetch.py", line 58, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/tmp/ipykernel_1028044/3003076598.py", line 30, in __getitem__
    coef_c, coef_d = getTrueCoef(label_list, time_list, features.shape[0], len(cL) )
  File "/tmp/ipykernel_1028044/4275206083.py", line 25, in getTrueCoef
    res_conf[inds, c] = 1
IndexError: index 2919 is out of bounds fo

(2700, 8576)
(2700, 7)
(2700, 7)


(2811, 8576)
(2811, 7)
(2811, 7)


(2700, 8576)
(2700, 7)
(2700, 7)


(2700, 8576)
(2700, 7)
(2700, 7)


(2700, 8576)
(2700, 7)
(2700, 7)


(2797, 8576)
(2797, 7)
(2797, 7)


(2700, 8576)
(2700, 7)
(2700, 7)


(2933, 8576)
(2933, 7)
(2933, 7)


(2700, 8576)
(2700, 7)
(2700, 7)


(2700, 8576)
(2700, 7)
(2700, 7)


(2700, 8576)
(2700, 7)
(2700, 7)


(2720, 8576)
(2720, 7)
(2720, 7)


(2700, 8576)
(2700, 7)
(2700, 7)


(2700, 8576)
(2700, 7)
(2700, 7)


(2700, 8576)
(2700, 7)
(2700, 7)


(2700, 8576)
(2700, 7)
(2700, 7)


(2819, 8576)
(2819, 7)
(2819, 7)


(2700, 8576)
(2700, 7)
(2700, 7)


(2911, 8576)
(2911, 7)
(2911, 7)


(2712, 8576)
(2712, 7)
(2712, 7)


(2700, 8576)
(2700, 7)
(2700, 7)


(3036, 8576)
(3036, 7)
(3036, 7)


(2700, 8576)
(2700, 7)
(2700, 7)


(2839, 8576)
(2839, 7)
(2839, 7)


(2700, 8576)
(2700, 7)
(2700, 7)


(2821, 8576)
(2821, 7)
(2821, 7)


(2700, 8576)
(2700, 7)
(2700, 7)


(2803, 8576)
(2803, 7)
(2803, 7)


(2700, 8576)
(2700, 

In [118]:
ind = np.array([True if (i >= 20 and i < 20 + 15) else False for i in range(100)])
print(ind)

val = np.random.uniform(size = 100)
print(val)
print(val[20])
print(val[ind])

[False False False False False False False False False False False False
 False False False False False False False False  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False]
[0.25329134 0.62364651 0.5522916  0.26783181 0.65075275 0.38129595
 0.93924415 0.28977164 0.35033819 0.83425069 0.26552837 0.98298266
 0.35538832 0.45862694 0.64324909 0.34391024 0.56295715 0.89339315
 0.73759986 0.88861315 0.51528868 0.46544004 0.2395967  0.53255697
 0.39856292 0.67898395 0.35864691 0.08793207 0.03769187 0.48948458
 0.94847351 0.49793608 0.27777819 0.2431348  0.95590225